In [1]:
import pytesseract as pt
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from pdf2image import convert_from_path

In [2]:
def area(approx):
    x = approx[:,0,0]
    y = approx[:,0,1]
    return (max(x)-min(x))*(max(y)-min(y))

def start_point(approx):
    x = approx[:,0,0]
    y = approx[:,0,1]
    return (min(x),max(y))

def end_point(approx):
    x = approx[:,0,0]
    y = approx[:,0,1]
    return (max(x),min(y))

def get_points(approx):
    x = approx[:,0,0]
    y = approx[:,0,1]
    return (min(x),max(x),min(y),max(y))

def get_all_points(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    coordinates = []
    for contour in contours:
        epsilon = 0.05 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        if len(approx) == 4:
            if (area(approx)>5000):
                coordinates.append(get_points(approx))
#                 print(area(approx))
#                 cv2.rectangle(image, start_point(approx),end_point(approx), (0, 255, 255), 5)
    return coordinates

def get_tables(image):
    coordinates = get_all_points(image)
#     print(len(coordinates))
    tables = []
    for coordinate in coordinates:
        tables.append(image[coordinate[2]:coordinate[3],coordinate[0]:coordinate[1]])
    return tables

def get_image_with_table(img):
    image = img
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    coordinates = []
    for contour in contours:
        epsilon = 0.05 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        if len(approx) == 4:
            if (area(approx)>5000):
                coordinates.append(get_points(approx))
#                 print(area(approx))
                cv2.rectangle(image, start_point(approx),end_point(approx), (0, 255, 255), 5)
    return image

In [ ]:
books_punjabi = sorted(['data_books/punjabi/'+book for book in os.listdir('data_books/punjabi/')])

In [3]:
book = 'data_books/punjabi/pp_kharif__pb_table.pdf'

In [4]:
images = convert_from_path(book,dpi=300)

In [5]:
tables = [get_tables(np.array(image)) for image in images]

In [ ]:
table_small = cv2.imread('table-small-2.png')
table_large = cv2.imread('table-large-2.png')

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(ass)

In [6]:
def get_cells_coordinates(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh1,img = cv2.threshold(img,128,255,cv2.THRESH_OTSU)
    img = cv2.bitwise_not(img)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, img.shape[1]//150))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (np.array(img).shape[1]//150, 1))

    vertical_lines = cv2.erode(img, vertical_kernel, iterations=5)
    vertical_lines = cv2.dilate(vertical_lines, vertical_kernel, iterations=5)

    horizontal_lines = cv2.erode(img, hor_kernel, iterations=30)
    horizontal_lines = cv2.dilate(horizontal_lines, hor_kernel, iterations=30)

    vertical_horizontal_lines = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)
    vertical_horizontal_lines = cv2.erode(~vertical_horizontal_lines, kernel, iterations=3)

    thresh, vertical_horizontal_lines = cv2.threshold(vertical_horizontal_lines,128,255,cv2.THRESH_OTSU)
    b_image = cv2.bitwise_not(cv2.bitwise_xor(img,vertical_horizontal_lines))

    contours, hierarchy = cv2.findContours(vertical_horizontal_lines, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    temp_bound_box = [cv2.boundingRect(c) for c in contours]
    (contours, temp_bound_box) = zip(*sorted(zip(contours, temp_bound_box),key=lambda x:x[1][1]))
    boundingBoxes=[]
    for box in temp_bound_box:
        print(box)
        boundingBoxes.append(box)
    return boundingBoxes

def get_text_csv(bounding_boxes,image):
    cols = len(np.unique(np.array(bounding_boxes)[:,0]))
    rows = len(np.unique(np.array(bounding_boxes)[:,1]))
    
    rows_start_points = sorted(np.unique(np.array(bounding_boxes)[:,1]))
    cols_start_points = sorted(np.unique(np.array(bounding_boxes)[:,0]))
    rows_end_points = sorted(np.unique(np.array(bounding_boxes)[:,1]+np.array(bounding_boxes)[:,3]))
    cols_end_points = sorted(np.unique(np.array(bounding_boxes)[:,0]+np.array(bounding_boxes)[:,2]))
    
    table = []
    for i in range(rows):
        table.append([])
        for j in range(cols):
            table[i].append(())

    for idx,box in enumerate(bounding_boxes):
        col_start, row_start, w, h = box
        row_end, col_end = row_start+h,col_start+w
        #     print(rows_start_points.index(row_start)-rows_end_points.index(row_end))
        for i in range(rows_start_points.index(row_start),rows_end_points.index(row_end)+1):
            for j in range(cols_start_points.index(col_start),cols_end_points.index(col_end)+1):
                table[i][j]=box
#                 print(i,j,idx)
#     display_cell(image,table[7][0])
    # text = []
    all_text = ''
    for idx,row in enumerate(table):
    #     text.append([])
        for cell in row:
            x, y, w, h = cell
            ocr_text = pt.image_to_string(image[y:y+h,x:x+w],config='--psm 6',
                                          lang='pan').replace('\n','').replace('\x0c','').replace('_','')
#             print(ocr_text)
            all_text = all_text+ocr_text+','
        #         text[idx].append(ocr_text)
        all_text=all_text+'\n'
    return all_text

In [ ]:
bounding_boxes = get_cells_coordinates(table_large)

In [7]:
def display_cell(image,bounding_box):
    x, y, w, h = bounding_box
    plt.imshow(image[y:y+h,x:x+w])
    print(pt.image_to_string(image[y:y+h,x:x+w],config='--psm 6 --oem 3',lang='pan'))

In [ ]:
plt.imsave('table-1cell.png',tables[210][0])

In [ ]:
ass = cv2.imread('table-1cell.png')

In [ ]:
bounding_boxes = get_cells_coordinates(ass)
ocr = get_text_tsv(bounding_boxes,ass)

In [ ]:
all_text = ''
for idx,table in enumerate(tables):
    all_text = all_text + f'page-{idx}\n'
    for curr in table:
        try:
            bounding_boxes = get_cells_coordinates(curr)
            ocr = get_text_csv(bounding_boxes,curr)
            all_text = all_text + ocr + '\n'
        except:
            continue
    all_text = all_text + '\n\n\n\n'

In [ ]:
output_file = 'data_books/ocr_punjabi/pp_kharifs_tables.txt'
f=open(output_file,'w')
f.write(all_text)
f.close()

In [ ]:
all_text = ''
for idx,table in enumerate(tables):
    all_text = all_text + f'page-{idx}\n'
    for curr in table:
        try:
            ocr = pt.image_to_string(curr,config='--psm 6',lang='pan')
            all_text = all_text + ocr + '\n'
        except:
            continue
    all_text = all_text + '\n\n\n\n'

In [ ]:
output_file = 'data_books/ocr_punjabi/pp_kharifs_notes.txt'
f=open(output_file,'w')
f.write(all_text)
f.close()

In [ ]:
plt.imshow(tables[5][0])

In [9]:
error_page = [5,6,9,11,12,13,14,20,22,23,24,29,30,32,33,39,41,49,50,59,60,62]
for i in range(len(error_page)):
    error_page[i]=error_page[i]-1
print(error_page)

[4, 5, 8, 10, 11, 12, 13, 19, 21, 22, 23, 28, 29, 31, 32, 38, 40, 48, 49, 58, 59, 61]


In [8]:
def get_cells_coordinates_include_verification(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh1,img = cv2.threshold(img,128,255,cv2.THRESH_OTSU)
    img = cv2.bitwise_not(img)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, img.shape[1]//150))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (np.array(img).shape[1]//150, 1))

    vertical_lines = cv2.erode(img, vertical_kernel, iterations=5)
    vertical_lines = cv2.dilate(vertical_lines, vertical_kernel, iterations=5)

    horizontal_lines = cv2.erode(img, hor_kernel, iterations=30)
    horizontal_lines = cv2.dilate(horizontal_lines, hor_kernel, iterations=30)

    vertical_horizontal_lines = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)
    vertical_horizontal_lines = cv2.erode(~vertical_horizontal_lines, kernel, iterations=3)

    thresh, vertical_horizontal_lines = cv2.threshold(vertical_horizontal_lines,128,255,cv2.THRESH_OTSU)
    b_image = cv2.bitwise_not(cv2.bitwise_xor(img,vertical_horizontal_lines))

    contours, hierarchy = cv2.findContours(vertical_horizontal_lines, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    temp_bound_box = [cv2.boundingRect(c) for c in contours]
    (contours, temp_bound_box) = zip(*sorted(zip(contours, temp_bound_box),key=lambda x:x[1][1]))
    boundingBoxes=[]
    for box in temp_bound_box:
#         print(box)
        boundingBoxes.append(box)
    print(boundingBoxes[0][2:],(np.shape(image)[1],np.shape(image)[0]))
    x = input()
    if (x=='n'):
        return boundingBoxes
    return boundingBoxes[1:]

In [ ]:
image = tables[5][0]
img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh1,img = cv2.threshold(img,128,255,cv2.THRESH_OTSU)
img = cv2.bitwise_not(img)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, img.shape[1]//150))
hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (np.array(img).shape[1]//150, 1))

In [ ]:
vertical_lines = cv2.erode(img, vertical_kernel, iterations=5)
vertical_lines = cv2.dilate(vertical_lines, vertical_kernel, iterations=5)

In [ ]:
horizontal_lines = cv2.erode(img, hor_kernel, iterations=30)
horizontal_lines = cv2.dilate(horizontal_lines, hor_kernel, iterations=30)

In [ ]:
vertical_horizontal_lines = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)
vertical_horizontal_lines = cv2.erode(~vertical_horizontal_lines, kernel, iterations=3)

In [ ]:
thresh, vertical_horizontal_lines = cv2.threshold(vertical_horizontal_lines,128,255,cv2.THRESH_OTSU)
b_image = cv2.bitwise_not(cv2.bitwise_xor(img,vertical_horizontal_lines))

In [10]:
table_to_be_edited = [tables[i] for i in error_page]

In [11]:
all_text = ''
for idx,table in enumerate(table_to_be_edited):
    all_text = all_text + f'page-{error_page[idx]}\n'
    print(len(table))
    for curr in table:
        try:
            bounding_boxes = get_cells_coordinates_include_verification(curr)
            ocr = get_text_csv(bounding_boxes,curr)
            all_text = all_text + ocr + '\n'
            print(ocr)
        except:
            continue
    all_text = all_text + '\n\n\n\n'

2
(1654, 283) (1654, 283)

“ਤੱਤ (ਕਿਲੋ ਪੁਤੀ ਏਕੜ),“ਤੱਤ (ਕਿਲੋ ਪੁਤੀ ਏਕੜ),“ਤੱਤ (ਕਿਲੋ ਪੁਤੀ ਏਕੜ),ਖਾਦਾਂ (ਕਿਲੋ ਪੁਤੀ ਏਕੜ),ਖਾਦਾਂ (ਕਿਲੋ ਪੁਤੀ ਏਕੜ),ਖਾਦਾਂ (ਕਿਲੋ ਪੁਤੀ ਏਕੜ),ਖਾਦਾਂ (ਕਿਲੋ ਪੁਤੀ ਏਕੜ),
ਨਾਈਟ੍ਰੋਜਨ,ਫ਼ਾਸਫ਼ੋਰਸ,ਪੋਟਾਸ਼,,ਡੀਏਪੀ,ਜਾਂ ਸਿੰਗਲ ਸੁਪਰਫਾਸਫੇਟ,ਮਿਊਰੇਟ ਆਫ਼ ਪੋਟਾਸ਼,
42,12,12,90,27,75,20,

(777, 125) (1649, 767)
n
ਜੈਵਿਕ ਖਾਦ,ਮਾਤਰਾ (ਟਨਪ੍ਰਤੀ ਏਕੜ),ਖਾਦ ਦੀ ਬਚਤ,
ਰੂੜੀ,6,16 ਕਿਲੋ ਨਾਈਟ੍ਰੋਜਨ (35 ਕਿਲੋ ਯੂਰੀਆ),
ਪ੍ਰੈਸਮੰਡ,6,25 ਕਿਲੋ ਨਾਈਟ੍ਰੋਜਨ (55 ਕਿਲੋ ਯੂਰੀਆ),
ਮੁਰਗੀਆਂ ਦੀ ਖਾਦ,2.5,25 ਕਿਲੋ ਨਾਈਟ੍ਰੋਜਨ (55 ਕਿਲੋ ਯੂਰੀਆ),
ਗੋਬਰ ਗੈਸ ਸਲੌਰੀ (ਸੁੱਕੀ),2.4,16 ਕਿਲੋ ਨਾਈਟ੍ਰੋਜਨ (35 ਕਿਲੋ ਯੂਰੀਆ),
ਪਰਾਲੀਚਾਰ,2.0,16 ਕਿਲੋ ਨਾਈਟ੍ਰੋਜਨ (35 ਕਿਲੋ ਯੂਰੀਆ)#,
ਹਰੀ ਖਾਦ,,25 ਕਿਲੋ ਨਾਈਟ੍ਰੋਜਨ ਤੱਤ (55 ਕਿਲੋ ਯੂਰੀਆ)##,
ਗਰਮੀ ਰੁੱਤ ਦੀ ਮੂੰਗੀ ਦੀ ਰਹਿੰਦ-ਖੂੰਹਦ,,16 ਕਿਲੋ ਨਾਈਟ੍ਰੋਜਨ (35 ਕਿਲੋ ਯੂਰੀਆ)##,

1
(1653, 352) (1653, 352)

ਕਿਸਮਾਂ,ਪਨੀਰੀ ਬੀਜਣ ਦਾ ਸਮਾਂ,ਲੁਆਈ ਦਾ ਸਮਾਂ,
ਪੰਜਾਬ ਬਾਸਮਤੀ 7, 5 ਅਤੇ ਪੂਸਾ ਬਾਸਮਤੀ1121, 1637 ਤੇ 1718,ਜੂਨ ਦਾ ਪਹਿਲਾ ਪੰਦਰ੍ਹਵਾੜਾ,ਜੁਲਾਈ ਦਾ ਪਹਿਲਾ ਪੰਦਰ੍ਹਵਾੜਾ,
ਸੀ ਐਸ ਆਰ 30 ਅਤੇ ਪੂਸਾ ਬਾਸਮਤੀ1509,ਜੂਨ ਦਾ ਦੂਜਾ ਪੰਦਰ੍ਹਵਾੜਾ,ਜੁਲਾਈ ਦਾ ਦੂਜਾ ਪੰਦਰ੍ਹਵਾੜਾ,

1
(1650, 501) (1650, 501)

ਜ਼


(154, 67) (154, 67)

੭,5,],,0,,0,
੭,5,],,0,,0,

(153, 67) (153, 67)

੭.,5,],

(206, 64) (210, 68)

(154, 67) (154, 67)

],2.,੭,੭,

(153, 67) (153, 67)

],23,੭,

(206, 64) (210, 68)

(154, 67) (154, 67)

322,322,322,ਰਿ,ਰਿ,
322,322,322,ਰਿ,ਰਿ,
322,322,322,ਰਿ,ਰਿ,
322,322,322,ਰਿ,ਰਿ,

(153, 67) (153, 67)

(206, 64) (210, 68)

(154, 67) (154, 67)

986.,986.,986.,986.,986.,986.,੨,੨,
986.,986.,986.,986.,986.,986.,੨,੨,
986.,986.,986.,986.,986.,986.,੨,੨,
986.,986.,986.,986.,986.,986.,੨,੨,
986.,986.,986.,986.,986.,986.,੨,੨,

(153, 67) (153, 67)

(206, 64) (210, 68)

ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,
ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,
ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,
ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,ਅੰਮ੍ਰਿਤਸਰ,

(154, 66) (154, 66)

ਪ,

(153, 66) (153, 66)

ਪ,

(297, 115) (297, 115)

(306, 115) (306, 115)

(337, 107) (371, 116)

(397, 107) (431, 116)

(209, 186) (209, 186)

1
(1657, 286) (1657, 286)

ਲਾਈਨਾਂ ਵਿਚਕਾਰਫ਼ਾਸਲਾ,ਪਹੀਏ ਦਾ ਅਕਾਰ (ਇੰਚਾਂ ਜਾਂ ਸੈਂਟੀਮੀਟਰ ਵਿੱਚ),ਪਹੀਏ ਦਾ ਅਕਾਰ (ਇੰਚਾਂ ਜ

In [ ]:
21 40

In [14]:
output_file = 'data_books/ocr_punjabi/pp_kharifs_tables_error_update.txt'
f=open(output_file,'w')
f.write(all_text)
f.close()